In [2]:
import matplotlib.pyplot as plt
import cv2
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from progressbar import ProgressBar
import time
from multiprocessing import Pool
from functools import partial

from sklearn.model_selection import train_test_split

In [3]:
dataset_dir = "C:/Users/frede/Desktop/Bachelorprojekt/datasets/VALID-dataset"

In [4]:
def find_image_from_json(json_path):
    f = open(json_path)
    json_file = json.load(f)
    path_to_image_file = json_file["file_name"]
    
    return(f"C:/Users/frede/Desktop/Bachelorprojekt/datasets/VALID-dataset/{path_to_image_file}")

def find_image_from_json_simple(json_path):
    f = open(json_path)
    json_file = json.load(f)
    path_to_image_file = json_file["file_name"]
    #print(path_to_image_file)
    
    return(os.path.join(dataset_dir,path_to_image_file))

In [5]:
#define a custom dataset class
class firstDataset(Dataset):
    def __init__(self, dataset_dir):
        self.dataset_dir = dataset_dir
        self.label_files = (os.listdir(os.path.join(dataset_dir, "label")))
        self.depth_files = (os.listdir(os.path.join(dataset_dir, "depth")))
        
    def __len__(self):
        return len(self.label_files)
    
    def __getitem__(self, idx):
        #path to image and depth image
        current_image_path = find_image_from_json_simple(os.path.join(self.dataset_dir, "label", self.label_files[idx]))
        current_depth_image_path = os.path.join(self.dataset_dir, "depth", self.depth_files[idx])
        
        #Read in image and depth image
        image = cv2.imread(current_image_path).astype(float)
        depth_image = cv2.imread(current_depth_image_path).astype(float)
        
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256,256)),
        ])
        
        image = transform(image)
        depth_image = transform(depth_image)
        
        return image,depth_image

In [6]:
device = torch.device("cuda")

In [7]:
#input size: amount_img, channels, width, height
def resize_256_256(inp):
    return(F.interpolate(inp, size=(256,256), mode="bilinear", align_corners=False))

In [1]:
def create_data_valid(path):
    start_time = time.time()
    images_list = []
    depth_images_list = []
    depth_images_no_rgb_list = []

    t = []
    ignore_indicies = list(range(2865,2928)) + list(range(2966,2972)) + list(range(2975,3029))
    ignore_indicies_strings = ["0"+str(element)+".png" for element in ignore_indicies]
    for i, (depth, label) in (enumerate(zip(os.listdir(os.path.join(path,"depth")), os.listdir(os.path.join(path,"label"))))):

        
        if depth not in ignore_indicies_strings:  
            print(f"appending {depth}")
            # Path to image and depth image
            t.append((i,depth))
            current_image_path = find_image_from_json_simple(os.path.join(path, "label", label))
            current_depth_image_path = os.path.join(path,"depth", depth)

            # Read in image and depth image

            image = cv2.imread(current_image_path).astype(float)   
            depth_image_r = cv2.imread(current_depth_image_path,-1)/256
            depth_image_r = np.clip(depth_image_r, 0.001, 80, depth_image_r)

            depth_image_r = np.expand_dims(depth_image_r, axis=2).repeat(3, axis=2)

            image = resize_256_256(torch.from_numpy(image).unsqueeze(0).permute(0,3,1,2)).squeeze(0)
            depth_image = resize_256_256(torch.from_numpy(depth_image_r).unsqueeze(0).permute(0,3,1,2)).squeeze(0)
            depth_image_no_rgb = (resize_256_256(torch.from_numpy(depth_image_r).unsqueeze(0).permute(0,3,1,2))[:,:1,:,:]).squeeze(0) # (1,1,256,256)
            
            # Add image tensor and depth image tensor to lists
            images_list.append(image.half())
            depth_images_list.append(depth_image.half())
            depth_images_no_rgb_list.append(depth_image_no_rgb.half())

            
        else:
            print(f"skipped {depth}")
        
           

    tensor_images = torch.stack(images_list)
    tensor_depth_images = torch.stack(depth_images_list)
    tensor_depth_images_no_rgb = torch.stack(depth_images_no_rgb_list)
    elapsed_time = time.time() - start_time
    
    
    #save tensors locally to free up memory
    if not os.path.exists("VALID_tensors"):
        os.makedirs("VALID_tensors")
    torch.save(tensor_images, "VALID_tensors/VALID_tensor_images.pt")
    torch.save(tensor_depth_images, "VALID_tensors/VALID_tensor_depth_images.pt")
    torch.save(tensor_depth_images_no_rgb, "VALID_tensors/VALID_tensor_depth_images_no_rgb.pt")
    
    return tensor_images,tensor_depth_images,tensor_depth_images_no_rgb #delete from memory

    
    #return(tensor_images,tensor_depth_images)
data = create_data_valid(dataset_dir)


NameError: name 'dataset_dir' is not defined